# Import packages

In [1]:
import sys
import pandas as pd
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
df = pd.read_csv('../inputs/inputs.csv', index_col=0)
df.head()

,byear,sex,ret_age,education,init_wage,pension,bal_rrsp,bal_tfsa,bal_other_reg,bal_unreg,cont_rate_rrsp,cont_rate_tfsa,cont_rate_other_reg,cont_rate_unreg,withdrawal_rrsp,withdrawal_tfsa,withdrawal_other_reg,withdrawal_unreg,replacement_rate_db,rate_employee_db,income_previous_db,init_dc,rate_employee_dc,rate_employer_dc,s_byear,s_sex,s_ret_age,s_education,s_init_wage,s_pension,s_bal_rrsp,s_bal_tfsa,s_bal_other_reg,s_bal_unreg,s_cont_rate_rrsp,s_cont_rate_tfsa,s_cont_rate_other_reg,s_cont_rate_unreg,s_withdrawal_rrsp,s_withdrawal_tfsa,s_withdrawal_other_reg,s_withdrawal_unreg,s_replacement_rate_db,s_rate_employee_db,s_income_previous_db,s_init_dc,s_rate_employee_dc,s_rate_employer_dc,weight,couple,prov,first_residence,second_residence,price_first_residence,price_second_residence,business,price_business,mix_bonds,mix_bills,mix_equity,fee,fee_equity,credit_card,personal_loan,student_loan,car_loan,credit_line,first_mortgage,second_mortgage,other_debt,credit_card_payment,personal_loan_payment,student_loan_payment,car_loan_payment,credit_line_payment,first_mortgage_payment,second_mortgage_payment,other_debt_payment,claim_age_cpp,s_claim_age_cpp,cap_gains_unreg,s_cap_gains_unreg,realized_losses_unreg,s_realized_losses_unreg,init_room_rrsp,init_room_tfsa,s_init_room_rrsp,s_init_room_tfsa
0,1992.0,female,55.0,university,15000.0,NaN,NaN,8700.000000,NaN,NaN,NaN,0.080000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5210.0,False,on,350000.0,NaN,350000.0,NaN,NaN,NaN,0.000000,1.000000,0.000000,0.003150,0.000000,700.0,NaN,NaN,7200.0,NaN,205000.0,NaN,NaN,700.0,NaN,NaN,600.0,NaN,1300.0,NaN,NaN,60.0,NaN,0,0,0,0,0,0,NaN,NaN
1,1985.0,male,65.0,university,10000.0,NaN,NaN,NaN,NaN,40000.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5210.0,False,on,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.833333,0.166667,0.012500,0.004167,NaN,NaN,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,65.0,NaN,0,0,0,0,0,0,NaN,NaN
2,1988.0,male,65.0,post-secondary,79000.0,NaN,17050.359712,5683.453237,NaN,NaN,0.00000,0.000000,NaN,NaN,0.0,0.0,NaN,NaN,0.7,0.05,NaN,NaN,NaN,NaN,1988.0,female,60.0,university,60000.0,NaN,12949.640288,4316.546763,NaN,NaN,0.00000,0.000000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.04,0.03,1160.0,True,on,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,0.009231,0.000000,64.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,200.0,NaN,NaN,NaN,NaN,NaN,65.0,60.0,0,0,0,0,0,0,0.0,0.0
3,1985.0,male,70.0,post-secondary,60000.0,NaN,NaN,428.571429,NaN,5714.285714,NaN,0.004762,NaN,0.047619,NaN,0.0,NaN,0.0,NaN,NaN,NaN,1000.0,0.04,0.04,1993.0,female,70.0,high school,45000.0,NaN,NaN,321.428571,NaN,4285.714286,NaN,0.004762,NaN,0.047619,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1160.0,True,on,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,0.003333,0.000000,2500.0,5000.0,10000.0,NaN,NaN,NaN,NaN,NaN,200.0,100.0,300.0,NaN,NaN,NaN,NaN,NaN,70.0,70.0,0,0,0,0,0,0,0.0,0.0
4,1988.0,female,65.0,university,85000.0,NaN,17276.422764,11747.967480,NaN,1036.585366,0.04878,0.048780,NaN,0.012195,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1988.0,male,65.0,high school,38000.0,NaN,7723.577236,5252.032520,NaN,463.414634,0.04878,0.048780,NaN,0.012195,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1160.0,True,on,423000.0,NaN,423000.0,NaN,NaN,NaN,0.229885,0.425287,0.344828,0.008966,0.005172,NaN,NaN,NaN,10000.0,NaN,393000.0,NaN,NaN,NaN,NaN,NaN,800.0,NaN,1900.0,NaN,NaN,65.0,65.0,0,0,0,0,0,0,0.0,0.0


# Create fake dataset

In [3]:
size = 5000
inputs = pd.DataFrame(np.nan, index=range(size), columns=df.columns)

def random_values(vars, size):
    indices = np.random.choice(df.index, size)
    return df.loc[indices, vars].values


# create first spouse
l_vars_p = ['sex', 'education', 'init_wage', 'pension',
            ['byear', 'ret_age','replacement_rate_db', 'rate_employee_db',
             'income_previous_db'],
            ['bal_rrsp', 'cont_rate_rrsp', 'withdrawal_rrsp'], 
            ['bal_tfsa', 'cont_rate_tfsa', 'withdrawal_tfsa'],
            ['bal_other_reg', 'cont_rate_other_reg', 'withdrawal_other_reg'],
            ['bal_unreg', 'cont_rate_unreg', 'withdrawal_unreg'],
            ['init_dc', 'rate_employee_dc', 'rate_employer_dc'],
            'claim_age_cpp', 'cap_gains_unreg', 'realized_losses_unreg',
            'init_room_rrsp', 'init_room_tfsa']

for vars in l_vars_p:
    inputs[vars] = random_values(vars, size)

# create variables for household
inputs.weight = 1

l_vars_hh = ['couple', 'prov', 
             ['first_residence', 'price_first_residence', 'first_mortgage', 
              'first_mortgage_payment'],
             ['second_residence', 'price_second_residence', 'second_mortgage',
              'second_mortgage_payment'],
             ['business', 'price_business'],
             ['mix_bonds', 'mix_bills', 'mix_equity', 'fee', 'fee_equity'],
             ['credit_card', 'credit_card_payment'],
             ['personal_loan', 'personal_loan_payment'],
             ['student_loan', 'student_loan_payment'],
             ['car_loan', 'car_loan_payment'],
             ['credit_line', 'credit_line_payment'],
             ['other_debt', 'other_debt_payment']]

for vars in l_vars_hh:
    inputs[vars] = random_values(vars, size)
             
# create spouse for couples
l_vars_sp = []
for vars in l_vars_p:
    if type(vars) == list:
        l_vars_sp.append(['s_' + var for var in vars])
    else:
        l_vars_sp.append('s_' + vars)
        
for s_vars, vars in zip(l_vars_sp, l_vars_p):
    inputs[s_vars] = random_values(vars, size)
    inputs.loc[~inputs.couple, s_vars] = np.nan
    
inputs.loc[inputs.couple & (inputs.sex == 'female'), 's_sex'] = 'male'
inputs.loc[inputs.couple & (inputs.sex == 'male'), 's_sex'] = 'female'

# not retired: byear + ret_age > 2018
inputs.ret_age = np.maximum(2019 - inputs.byear, inputs.ret_age)
inputs.s_ret_age = np.maximum(2019 - inputs.s_byear, inputs.s_ret_age)

In [4]:
# RPP DB: need to be run to avoid errors when running simulator
## upper bound on contribution (B. Morency: employee+employer could be > 0.18):
inputs.rate_employee_db.clip(upper=0.09, inplace=True)
inputs.s_rate_employee_db.clip(upper=0.09, inplace=True)

## clip replacement rate (min: 2% per remaining year of work; 
# max: min(2% per year of work from 18 to retirement, 70%))
def clip_replacement_rate(row):
    low_b = min(0.02 * (row['ret_age'] - (2018 - row['byear'])), 0.70)
    up_b = min(0.02 * (row['ret_age'] - 18), 0.70)
    return np.clip(row['replacement_rate_db'], low_b, up_b)
   
inputs.replacement_rate_db = inputs[inputs.replacement_rate_db >= 0] \
    .apply(clip_replacement_rate, axis=1)

def s_clip_replacement_rate(row):
    low_b = min(0.02 * (row['s_ret_age'] - (2018 - row['s_byear'])), 0.70)
    up_b = min(0.02 * (row['s_ret_age'] - 18), 0.70)
    return np.clip(row['s_replacement_rate_db'], low_b, up_b)
   
inputs.s_replacement_rate_db = inputs[inputs.s_replacement_rate_db >= 0] \
    .apply(s_clip_replacement_rate, axis=1)

In [5]:
# save dataset
inputs.to_csv("../CPR/data/inputs/synth_inputs.csv")